This scripts depends on the generatation of envelope features that needs to be generated first.

In [12]:
#filename where you want to store the audio features
filename = 'script-2-N95_Audio_Features_PEF.pickle'

In [13]:
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import matplotlib
import matplotlib.pyplot as plt
import latexify as lt
import os
#from thinkdsp import read_json
import scipy
import speechpy
#LOSO Validation
loo = LeaveOneOut()
import tsfel
#for filter
from scipy.signal import butter
from scipy.signal import lfilter
from scipy.signal import freqz;
import pickle
import json

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='highpass', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y;

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
#the file where all data is arranged.
data = pd.read_csv("data_FVC_N95.csv")
print("Total Data Points = ",len(data))
data.head(25)

Total Data Points =  60


,Unnamed: 0,Name,Filename,rPEF,rFEV1,rFVC,rRatio,gPEF,gFEV1,gFVC,gRatio
0,NaN,P0,P0.FVC_N95.LCL.2dbk4ib2.wav,0.17,0.01,0.31,3.225806,4.10,2.89,3.00,96.333333
1,NaN,P1,P1.FVC_N95.2blk7k0f.wav,2.77,0.28,2.78,10.071942,7.98,3.33,3.76,88.563830
2,NaN,P2,P2.FVC_N95.2blk7k0f.wav,2.24,0.18,2.31,7.792208,7.98,3.33,3.76,88.563830
3,NaN,P3,P3.FVC_N95.LCL.amiramir.wav,0.16,0.01,0.35,2.857143,5.96,2.77,2.96,93.581081
4,NaN,P4,P4.FVC_N95.RCL.2dbj63ol.wav,1.22,0.11,1.59,6.918239,6.58,3.98,4.12,96.601942
5,NaN,P5,P5.FVC_N95.LCL.2dbjkojc.wav,0.72,0.07,0.76,9.210526,5.69,2.96,3.19,92.789969
6,NaN,P6,P6.FVC_N95.RCL.2dbha71f.wav,0.31,0.03,0.61,4.918033,5.40,2.96,3.45,85.797101
7,NaN,P9,P9.FVC_N95.2bldi9ts.wav,1.62,0.04,1.80,2.222222,8.42,3.36,3.60,93.333333
8,NaN,P10,P10.FVC_N95.2bldi9ts.wav,2.00,0.09,4.49,2.004454,8.42,3.36,3.60,93.333333
9,NaN,P11,P11.FVC_N95.2b66msvi.wav,2.41,0.15,3.45,4.347826,4.06,2.04,3.94,51.776650


In [14]:
#load time series feature of the envelope
    
with open('script-1-ts-fel-n95-Envelope.pickle', 'rb') as handle:
    ts_features_envelope = pickle.load(handle)

Audio_Features = {}

try: 
    measured = pd.read_csv('data-forced-breathing-1/GT_FB_N95.csv', index_col='Unnamed: 0')
except:
    measured = pd.read_csv('data-forced-breathing-1/GT_FB_N95.csv')
    
def get_features(filepath, lungParam):
    
    #use the below line if the data is in json
    #wave = read_json(filepath=filepath)
    
    #use the below line if the data is in wav
    if (filepath).endswith('.wav'):
        ys, fs = librosa.load(filepath, sr=16000)
    else:
        with open(filepath) as fp:
            input_data = json.load(fp)
        ys = np.array(input_data['payload']['values'])
        fs = 1000/input_data['payload']['interval_ms']
    
    if lungParam == 'FEV1':
        #print("FEV1 is the param. Audio signal will be clipped to peak only")
        peakIndex = np.where(ys == ys.max())[0][0]
        ys = ys[0:peakIndex+1];

    ys = speechpy.processing.preemphasis(ys, cof=0.98)
    
    
    #staching frames
    frames = speechpy.processing.stack_frames(ys, sampling_frequency=fs, frame_length=0.020, frame_stride=0.01, filter=lambda x: np.ones((x,)),zero_padding=True)

    power_spectrum = np.mean(speechpy.processing.power_spectrum(frames, fft_points=512), axis=0)
    
    
    mfcc = speechpy.feature.mfcc(ys, sampling_frequency=fs, frame_length=0.020, frame_stride=0.01,
             num_filters=40, fft_length=256, low_frequency=0, high_frequency=None)
    mfcc_cmvn = np.mean(speechpy.processing.cmvnw(mfcc,win_size=301,variance_normalization=True), axis=0)
    
    logenergy = speechpy.feature.lmfe(ys, sampling_frequency=fs, frame_length=0.020, frame_stride=0.01,
             num_filters=40, fft_length=256, low_frequency=0, high_frequency=None)
    
    logenergy = np.mean(logenergy, axis=0)
    
    mfe = speechpy.feature.mfe(ys, sampling_frequency=fs, frame_length=0.020, frame_stride=0.01,
            num_filters=40, fft_length=512, low_frequency=0, high_frequency=None)
    mfe = np.mean(mfe[0],axis=0)
    
    
    #get the features of the HT envelope of audio
    stat_features_envelope = ts_features_envelope[filepath.split('/')[-1]]
    
    
    cfg = tsfel.get_features_by_domain()
    stat_features = np.array(tsfel.time_series_features_extractor(cfg, ys, fs=16000, window=500))[0]
    
    #fc_coefficients = np.mean(librosa.feature.mfcc(y=ys, n_mfcc=13, sr=16000),axis=1)
    #print(mfc_coefficients.shape)
    
    
    melspectogram = np.mean(librosa.feature.melspectrogram(y=ys, sr=16000, n_mels=64, fmax=8000),axis=1)
    #print(melspectogram.shape)
    
    stft_spectrogram=np.abs(librosa.stft(ys))
    chromagram=np.mean(librosa.feature.chroma_stft(S=stft_spectrogram, sr=16000),axis=1)
    
    
    S,phase = librosa.magphase(librosa.stft(ys))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(S=S, sr=fs), axis=1) 
    zcr = np.mean(librosa.feature.zero_crossing_rate(ys)[0])
    
    
    feature_matrix_r = np.hstack((stat_features,stat_features_envelope,mfe, logenergy, mfcc_cmvn,power_spectrum,
                                  melspectogram,chromagram,spectral_rolloff,zcr))
    
    
    
    
    return feature_matrix_r


def load_data(lungParam, sex, LH, file, use_height=False):
    '''
    feature: it is either 'FEV1', 'FVC' or 'PEF'
    '''
    data = pd.read_csv(file)
    
    if sex == 'M' or sex == 'F':
        data = data[data['Sex'] == sex]
    if LH == 'Y' or LH == 'N':
        data = data[data['LH'] == LH]
    
    
    #get the ground truth
    y=  np.array([i for i in data['g'+lungParam]])
    
    #prepare to store a lot of features in X
    X=[]
    count = 0
    for file in data['Filename']:
        features = get_features("data-forced-breathing-1/N95/"+file, lungParam)
        #print("Size of features = ",features.shape)
        if use_height:
            uid = data.loc[data['Filename'] == file, 'Name'].values[0]
            height = measured.loc[measured['UID'] == uid, 'height'].values[0]
            features = np.append(features, height)
        Audio_Features[file] = features
        X.append(features)
        count += 1
    X =  np.array(X)
    
    
    #add the estiamted lung param as a feature
    estiamtedValues = np.atleast_2d(data['r'+lungParam].to_numpy()).T
    X = np.hstack((X, estiamtedValues))
    # Return arrays to plug into sklearn's cross-validation algorithms
    return X, np.array(y)

In [15]:
X, Y = load_data('PEF', 'A', 'A', "data_FVC_N95.csv", use_height=False)

588 94265 320 160.0
*** Feature extraction started ***
60


577 92565 320 160.0
*** Feature extraction started ***
60


635 101821 320 160.0
*** Feature extraction started ***
60


559 89731 320 160.0
*** Feature extraction started ***
60


563 90298 320 160.0
*** Feature extraction started ***
60


...

942 150937 320 160.0
*** Feature extraction started ***
60


787 126141 320 160.0
*** Feature extraction started ***
60


484 77698 320 160.0
*** Feature extraction started ***
60


628 100794 320 160.0
*** Feature extraction started ***
60


674 108019 320 160.0
*** Feature extraction started ***
60


In [16]:
X.shape, Y.shape

((60, 1209), (60,))

In [17]:
with open(filename, 'wb') as handle:
    pickle.dump(Audio_Features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
#testing area

cfg = tsfel.get_features_by_domain()

In [12]:
for key1 in cfg:
    #print("Key1 = ", key1)
    for key2 in cfg[key1]:
        print("\nKey2 = ", key2)
        for key3 in cfg[key1][key2]:
            if key3 == 'n_features':
                print("\tKey3 = ", cfg[key1][key2]['n_features'])


Key2 =  FFT mean coefficient
	Key3 =  nfreq

Key2 =  Fundamental frequency
	Key3 =  1

Key2 =  Human range energy
	Key3 =  1

Key2 =  LPCC
	Key3 =  n_coeff

Key2 =  MFCC
	Key3 =  num_ceps

Key2 =  Max power spectrum
	Key3 =  1

Key2 =  Maximum frequency
	Key3 =  1

Key2 =  Median frequency
	Key3 =  1

Key2 =  Power bandwidth
	Key3 =  1

Key2 =  Spectral centroid
	Key3 =  1

Key2 =  Spectral decrease
	Key3 =  1

Key2 =  Spectral distance
	Key3 =  1

Key2 =  Spectral entropy
	Key3 =  1

Key2 =  Spectral kurtosis
	Key3 =  1

Key2 =  Spectral positive turning points
	Key3 =  1

Key2 =  Spectral roll-off
	Key3 =  1

Key2 =  Spectral roll-on
	Key3 =  1

Key2 =  Spectral skewness
	Key3 =  1

Key2 =  Spectral slope
	Key3 =  1

Key2 =  Spectral spread
	Key3 =  1

Key2 =  Spectral variation
	Key3 =  1

Key2 =  Wavelet absolute mean
	Key3 =  widths

Key2 =  Wavelet energy
	Key3 =  widths

Key2 =  Wavelet entropy
	Key3 =  1

Key2 =  Wavelet standard deviation
	Key3 =  widths

Key2 =  Wavelet vari